In [19]:
import pandas as pd
import os

In [29]:
path = '/content/drive/MyDrive/2022_WeatherContest_Data/DataSet/fct_weather_data/2012'
area_list = os.listdir(path)

area_dic ={key:list() for key in area_list}

for area in area_list:
  file_list = os.listdir(path+'/'+area+'/')
  for file in file_list:
    if '_3시간기온_' in file:
      res = pd.read_csv(path+'/'+area+'/'+file,encoding='cp949')
      area_dic[area].append(res)

In [30]:
for area in area_list:
  length = len(area_dic[area])
  for i in range(length):

    # 컬럼명 변경
    df=area_dic[area][i] # 약한 복사 (원본 데이터도 바뀜, 이름 편하게 쓰려고)
    df.columns = ['day','hour','forecast','val']

    # 'day' 컬럼에 Start 란 문자열이 들어간 행(월 구분 행) index 추출 
    idxs = df[df['day'].str.contains('Start')].index

    # 'month' 컬럼 추가 
    start_num = 0
    month = 1 
    df['month'] = 0
    for end_num in idxs:
      df.loc[start_num:end_num,'month']= str(month)
      month += 1
      start_num = end_num
    df.loc[start_num:,'month']= month

    # 'day' 컬럼에 Start 란 문자열이 들어간 행 삭제
    df.drop(idxs,axis=0,inplace=True)

    # 'hour' 컬럼은 현재 float 형(오전 2시는 200, 오후 2시는 1400 이래 표기중)
    # 'hour' 을 100으로 나눠서 최대 10의 자리 정수로 변환
    df.hour = df.hour//100

    # 'hour' 과 'forecast' 의 합이 9, 12, 15, 33, 36, 39 인 행만 추출 살림
    condition1 = (df['hour']+df['forecast']!=9)
    condition2 = (df['hour']+df['forecast']!=12)
    condition3 = (df['hour']+df['forecast']!=15)
    condition4 = (df['hour']+df['forecast']!=33)
    condition5 = (df['hour']+df['forecast']!=36)
    condition6 = (df['hour']+df['forecast']!=39)
    drop_idxs = df.loc[condition1 & condition2 & condition3 & condition4 & condition5 & condition6].index
    df.drop(drop_idxs,axis=0,inplace =True)

    # 'when'컬럼 추가 예보 대상 시점 정보를 담음
    # 예보시점은 불필요
    df['when'] = df['hour']+df['forecast']
    df.drop('hour',axis=1,inplace =True)

    new_columns = ['month','day','when','forecast','val']
    area_dic[area][i] = df[new_columns]
    del new_columns
    # 달,일,예보대상시점,몇시간전 예보인지 순으로 재정렬 & 인덱스 재설정
    df.sort_values(['month','day','when','forecast'],inplace=True)
    df.reset_index(drop=True, inplace=True)

In [31]:
# regional_integration_df 딕셔너리 안에 지역별 outer join 된 dataframe이 들어감
# 딕셔너리 키 값은 지역명
regional_integration_df = {key:0 for key in area_list}

for area in area_list:
  length = len(area_dic[area])
  res = area_dic[area][0].copy()
  for i in range(1,length):
    res = pd.merge(res,area_dic[area][i],how='outer',on=['month','day','when','forecast'])
  regional_integration_df[area]=res

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: Passing 'suffixes' which cause duplicate columns {'val_x'} in the result is deprecated and will raise a MergeError in a future version.
  if __name__ == '__main__':


In [32]:
# 관측소마다 내놓은 예보 값의 mean과 max를 구함
for area in area_list:  
  val_mean = regional_integration_df[area].iloc[:,4:].mean(axis=1)
  val_max = regional_integration_df[area].iloc[:,4:].max(axis=1)
  regional_integration_df[area]['val_mean'] = val_mean
  regional_integration_df[area]['val_max'] = val_max

  # 사용할 컬럼만 추림 (['month','day','when','forecast','val_mean','val_max'])
  regional_integration_df[area] = regional_integration_df[area][['month','day','when','forecast','val_mean','val_max']]

In [33]:
regional_integration_df['광주']

,month,day,when,forecast,val_mean,val_max
1,1,1,9.0,7.0,1.0,1.0
2,1,1,12.0,10.0,-0.9,-0.9
3,1,1,15.0,13.0,-1.9,-1.9
9,1,1,33.0,31.0,2.0,2.0
10,1,1,36.0,34.0,0.0,0.0
...,...,...,...,...,...,...
40238,10,31,36.0,16.0,8.0,8.0
40239,10,31,39.0,19.0,6.0,6.0
40254,10,31,33.0,10.0,10.0,10.0
40255,10,31,36.0,13.0,8.0,8.0
